In [1]:

import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(42)
num_samples = 1000

taxa_juros = np.random.uniform(0, 15, num_samples)
taxa_cambio = np.random.uniform(1, 5, num_samples)
producao_industrial = np.random.uniform(50, 200, num_samples)

In [3]:
pib = 2 * taxa_juros + 3 * taxa_cambio + 0.5 * producao_industrial + np.random.normal(0, 5, num_samples)
inflacao = 0.5 * taxa_juros + 2 * taxa_cambio + 0.2 * producao_industrial + np.random.normal(0, 2, num_samples)
taxa_desemprego = -0.1 * taxa_juros + 0.3 * taxa_cambio + 0.4 * producao_industrial + np.random.normal(0, 1, num_samples)

In [5]:
df_dsa = pd.DataFrame({'taxa_juros': taxa_juros,
                       'taxa_cambio': taxa_cambio,
                       'producao_industrial': producao_industrial,
                       'pib': pib,
                       'inflacao': inflacao,
                       'taxa_desemprego': taxa_desemprego})

df_dsa.head()

,taxa_juros,taxa_cambio,producao_industrial,pib,inflacao,taxa_desemprego
0,5.618102,1.740532,89.255853,66.385407,23.102492,33.626470
1,14.260715,3.167604,87.046820,84.632680,30.193113,34.390431
2,10.979909,4.491783,185.938187,131.822108,52.517792,74.375747
3,8.979877,3.928900,87.431930,66.632640,29.988437,33.580560
4,2.340280,4.226245,90.792459,68.815242,26.593498,37.755042


In [6]:
X = df_dsa[['taxa_juros', 'taxa_cambio', 'producao_industrial']]
y = df_dsa[['pib', 'inflacao', 'taxa_desemprego']]

In [7]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

## Construção do Modelo Multi-Target

A Regressão Multi-Target, ou Regressão Multi-Saída, é uma abordagem usada para prever múltiplas variáveis dependentes simultaneamente. Em vez de prever uma única variável de saída, o modelo faz previsões para diversas variáveis de saída. Essa abordagem pode ser vantajosa quando há correlações entre as variáveis de saída, permitindo que o modelo capture essas dependências e forneça previsões mais precisas.

Existem várias maneiras de implementar a Regressão Multi-Target:

**Modelos Independentes**: Treinar um modelo de regressão separado para cada variável de saída. Isso é simples, mas ignora possíveis correlações entre as variáveis de saída.

**MultiOutputRegressor**: Utiliza um modelo de regressão base e estende para múltiplas saídas. Cada saída é tratada como uma regressão separada, mas o mesmo algoritmo de regressão é usado para todas as saídas. Por exemplo, MultiOutputRegressor(RandomForestRegressor(...)).

**Modelos Multivariados**: Alguns algoritmos de regressão são naturalmente capazes de lidar com múltiplas saídas, como Redes Neurais e alguns métodos de aprendizado profundo. Esses modelos podem capturar melhor as dependências entre as variáveis de saída.

**Modelos de Regressão Conjunta**: Utilizar uma abordagem que trata todas as variáveis de saída simultaneamente em um único modelo. Por exemplo, o algoritmo PLS (Partial Least Squares) pode ser usado para regressão multi-target.

In [9]:
modelo_dsa = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, random_state = 42))

O código acima define um modelo de regressão multi-saída usando uma combinação do MultiOutputRegressor e do RandomForestRegressor.

**RandomForestRegressor(n_estimators = 100, random_state = 42)**: Este é o modelo de regressão base que será utilizado. Ele consiste em uma floresta de 100 árvores de decisão (indicadas por n_estimators = 100). O parâmetro random_state = 42 é usado para garantir a reprodutibilidade dos resultados, fixando a semente do gerador de números aleatórios.

**MultiOutputRegressor(...)**: Esta classe permite estender um modelo de regressão para lidar com múltiplas variáveis dependentes (multi-saída). Cada saída é tratada como uma regressão separada.

Portanto, modelo_dsa é um modelo que utiliza uma floresta aleatória para prever múltiplas variáveis dependentes simultaneamente.

In [10]:
modelo_dsa.fit(X_treino_scaled, y_treino)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [11]:
y_pred = modelo_dsa.predict(X_teste_scaled)

In [12]:
mse = mean_squared_error(y_teste, y_pred, multioutput = 'raw_values')
print("Mean Squared Error de cada variável alvo:", mse)

Mean Squared Error de cada variável alvo: [38.86388322  4.93906348  1.20767905]


O Mean Squared Error (MSE) é uma métrica de avaliação usada para medir a diferença entre os valores previstos por um modelo e os valores reais observados.  O MSE calcula a média dos quadrados dos erros, ou seja, a média das diferenças quadráticas entre os valores reais e previstos. Um MSE menor indica um modelo mais preciso.

In [13]:
# Coeficiente de determinação
r2 = r2_score(y_teste, y_pred, multioutput = 'raw_values')
print("R2 de cada variável alvo:", r2)

R2 de cada variável alvo: [0.94231126 0.94961794 0.99611608]


O coeficiente de determinação, também conhecido como 𝑅2, é uma métrica que avalia a proporção da variabilidade da variável dependente que é explicada pelo modelo. Ele é amplamente usado para medir a qualidade do ajuste de modelos de regressão. 
O 𝑅2 varia entre 0 e 1, onde:

- 0 indica que o modelo não explica nenhuma variabilidade da variável alvo.
- 1 indica que o modelo explica toda a variabilidade da variável alvo.

In [15]:
resultados = pd.DataFrame(y_teste, columns = ['pib', 'inflacao', 'taxa_desemprego'])
resultados['pib_pred'] = y_pred[:, 0]
resultados['inflacao_pred'] = y_pred[:, 1]
resultados['taxa_desemprego_pred'] = y_pred[:, 2]
resultados.head()

,pib,inflacao,taxa_desemprego,pib_pred,inflacao_pred,taxa_desemprego_pred
521,63.014526,26.680746,30.276211,54.654841,25.079532,28.152093
737,101.764482,36.175549,52.482609,96.803324,35.607149,51.887139
740,70.147008,30.863583,40.719110,78.477116,31.769310,41.675455
660,80.842988,33.689196,50.421798,83.755337,34.332910,49.269336
411,64.364788,24.215227,28.111229,73.112515,23.484329,28.844566
